In [2]:
"""Track trading costs and commissions in your portfolio

Demonstrates how to track commissions and fees separately from trade costs.

Attributes
----------
instruments
transactions
portfolio groups
properties
cocoon
sub-holding keys
transaction configuration
"""

'Track trading costs and commissions in your portfolio\n\nDemonstrates how to track commissions and fees separately from trade costs.\n\nAttributes\n----------\ninstruments\ntransactions\nportfolio groups\nproperties\ncocoon\nsub-holding keys\ntransaction configuration\n'

# Track trading costs and commissions in your portfolio

The aim of this notebook is to show how LUSID can be used to track trading comissions in your portfolio. As a portfolio manager, you might want to track commissions and fees seperate from the trade cost itself. This seperation allows you to (a) track trading commissions against other portfolios, and (b) to measure your P&L and performance inclusive or exclusive of comissions. 

<br>

Consider the example below. Here we might want to track the "Trade Cost" seperate from the "Fees" and "Net money on trade". LUSID has functionality which allows us to easily store all these data points.


<img src="img/trade-comms/trade_cost.PNG" alt="Drawing" style="width: 800px;"/>

### Table of contents

1. [Setup client and scope](#setup)<br>
2. [Read in transactions data CSV file](#trd_csv_read)<br>
3. [Set up mapping for upserting instruments, portfolios, and transactions](#trd_map)<br>
4. [Set up the instrument master](#trd_im)<br>
5. [Create property definition for a sub-holding key to track commissions ](#prop_def)<br>
6. [Create the portfolio](#trd_port)<br>
7. [Upsert your transactions](#upsert_trd)<br>
8. [Create new transaction types to capture cash versus stock versus comissions](#trd_movements)<br>
9. [Create holdings report](#holdings)<br>
10. [View and track your comissions](#com_report)<br>


## (1) Setup client and scope  <a id="setup"></a>  

Import lusid and non-lusid packages, validate client and define scope.

In [2]:
# Import the general purpose Python packages

import copy
import os

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools import cocoon as lpt
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import format_instruments_response, format_portfolios_response, format_transactions_response
import lusid
import lusid.models as models
import globalfund as global_fund_tools

# Import data wrangling packages
# One we havent used in other notebooks is the pandas_utils package
# We use this to flatten the results of a get_holdings response to parse the data into a neat DataFrame

from IPython.display import display_html
import pandas as pd
import json
pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# define a scope and a porfolio to to used in this notebook

scope = "custodian"
portfolio = "LUSID_COM001"

## (2) Read in transactions data from file <a id="trd_csv_read"></a>  

First we need to load some trades into a pandas DataFrame from a trades CSV file. For the purposes of this example, we use 20 hypothetical trades across 10 seperate FTSE100 equities.

In [3]:
# The custodian file contains a "portfolio code" value
# For the purposes of this notebook, we also allow users of the notebook to pass their own "portfolio" value

cust_trades_df = pd.read_csv("data/trade-comms/custodian_trades.csv")
cust_trades_df["fund_code"] = portfolio
cust_trades_df.head(2)

,fund_code,fund_name,isin,client_id,security_description,security_group,trd_number,transaction_code,trade_date,settle_date,units,price,principal,trading_currency,country_of_risk,country_of_risk_name,trade_tax,broker_commission,exchange_rate
0,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0002162385,EQ_1234,Aviva,COMMON STOCK,trd_123,cust_buy,27/11/2019,29/11/2019,124626,4.01,499999.51,GBP,GB,United Kingdom,350,20,1
1,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB0002162385,EQ_1234,Aviva,COMMON STOCK,trd_124,cust_buy,27/11/2019,29/11/2019,12462,4.01,49997.54,GBP,GB,United Kingdom,350,30,1


## (3) Set up mapping <a id="trd_map"></a>  

Next we setup the mapping between LUSID and the external data source. At this point, it is worth noting that LUSID uses an [Extensible Data Model](https://support.finbourne.com/extensible-data-model). This means that we need to map the minimum fields required by LUSID's underlying business logic. Then all other external attributes get decorated into LUSID as properties.

In [4]:
mapping = {
    "cash_flag": {
        "cash_identifiers": {"security_group": ["CASH"]},
        "implicit": "trading_currency",
    },
    "instruments": {
        "identifier_mapping": {"ClientInternal": "client_id", "Isin": "isin"},
        "required": {"name": "security_description"},
        "properties": ["security_group", "country_of_risk", "country_of_risk_name",],
    },
    "transactions": {
        "identifier_mapping": {"ClientInternal": "client_id",},
        "required": {
            "code": "fund_code",
            "transaction_id": "trd_number",
            "type": "transaction_code",
            "transaction_price.price": "price",
            "transaction_price.type": "$Price",
            "total_consideration.amount": "principal",
            "units": "units",
            "transaction_date": "trade_date",
            "total_consideration.currency": "trading_currency",
            "settlement_date": "settle_date",
        },
        "optional": {"source": "$lusid_security_services"},
        "properties": ["exchange_rate", "trade_tax", "broker_commission",],
    },
    "portfolios": {
        "required": {
            "code": "fund_code",
            "display_name": "fund_name",
            "base_currency": "$GBP",
            "created": "$2018-01-01T00:00:00+00:00",
        }
    },
}

## (4) Set up the instrument master <a id="trd_im"></a>  

Next we setup the instrument master. To setup the instrument master, we need to extract unique instruments and their attributes from the transaction file. We also want to remove cash items as LUSID already has native data entities to represent cash. In other words, we don't need to create a seperate instrument master for cash.  For more details on Instruments in LUSID, please see our detailed and specific notebook on the topic [HERE](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/Instruments.ipynb).

In [5]:
# Create an instruments specific DataFrame for uploading

cust_trades_df_inst, mapping = lpt.identify_cash_items(
    cust_trades_df, mapping, remove_cash_items=True, file_type="instruments"
)

cust_trades_df_inst_unique = cust_trades_df_inst[["isin",
                                                  "client_id",
                                                  "security_description",
                                                  "security_group",
                                                  "country_of_risk",
                                                  "country_of_risk_name"
                                                 ]].drop_duplicates()
cust_trades_df_inst_unique

,isin,client_id,security_description,security_group,country_of_risk,country_of_risk_name
0,GB0002162385,EQ_1234,Aviva,COMMON STOCK,GB,United Kingdom
2,GB0000566504,EQ_1235,BHP,COMMON STOCK,GB,United Kingdom
4,GB0031348658,EQ_1236,Barclays,COMMON STOCK,GB,United Kingdom
6,GB0007980591,EQ_1237,BP,COMMON STOCK,GB,United Kingdom
8,GB0005405286,EQ_1238,HSBC,COMMON STOCK,GB,United Kingdom
10,GB0006043169,EQ_1239,Morrisons,COMMON STOCK,GB,United Kingdom
12,GB0008847096,EQ_1240,Tesco,COMMON STOCK,GB,United Kingdom
14,GB00BGDT3G23,EQ_1241,Rightmove,COMMON STOCK,GB,United Kingdom
16,GB00BH4HKS39,EQ_1242,vodafone,COMMON STOCK,GB,United Kingdom
18,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,GB,United Kingdom


Now that we have the instruments in a DataFrame, we can upload them into LUSID.

In [6]:
result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df_inst_unique,
    mapping_required=mapping["instruments"]["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=mapping["instruments"]["identifier_mapping"],
    property_columns=mapping["instruments"]["properties"],
    properties_scope=scope
)

succ, failed, errors = format_instruments_response(result)
print(f"number of successful upserts: {len(succ)}")
print(f"number of failed upserts    : {len(failed)}")
print(f"number of errors            : {len(errors)}")

number of successful upserts: 10
number of failed upserts    : 0
number of errors            : 0


Once the instruments have been upserted, we now have an instance of LUSID with an instrument master:

![Init](img/trade-comms/lusid_instruments.PNG)

## (5) Create the property definition  <a id="prop_def"></a>  

Next we create a property definition which will allow us to split out a holding into a number of sub-holdings (such as comission).<br>

Every property consist of a "{domain}/{scope}/{code}" configuration. This allows us to organise the data for easy access later. In the example below, we want to track asset type in the custodian scope on the transaction. Therefore the property is "Transaction/custodian/AssetType".

In [7]:
domain = "Transaction"
scope = "custodian"
code = "AssetType"

try:

    api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=lusid.models.CreatePropertyDefinitionRequest(
            domain = domain,
            scope = scope,
            code = code,
            value_required = None,
            display_name = "Type of asset movement",
            data_type_id = lusid.ResourceId(scope="system", code="string"),
            life_time = None
        )
    )
    
    print("Property created.")

except:
    try:
        prop_definition = (api_factory.build(lusid.api.PropertyDefinitionsApi).get_property_definition(
            domain = domain,
            scope = scope,
            code = code))
        print("Property already exists.")
        print("See defintion below")
        print("----------------------")
        print(prop_definition)
        
    except:
        print("Cannot find or create property")

Property created.


## (6) Create the portfolio <a id="trd_port"></a>  

Then we create a portfolio with the new property as a sub-holding key. These keys are used to organise a portfolio's holding from a set of transactions. You can read more about sub-holding keys [here](https://support.finbourne.com/what-are-subholding-keys).

In [8]:
result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df,
    mapping_required=mapping["portfolios"]["required"],
    mapping_optional={},
    file_type="portfolios",
    sub_holding_keys=["Transaction/custodian/AssetType"]
)

succ, failed = format_portfolios_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


Now we have an instrument master with a portfolio in LUSID.

![Init](img/trade-comms/lusid_port.PNG)

## (7) Upsert your transactions <a id="upsert_trd"></a>  

Next we upsert our transactions into LUSID. The process is simliar to upserting instruments and portfolios - we extract data from a CSV, complete some mappings, and then upsert to the LUSID APIs.

In [9]:
cust_trades_df, mapping = lpt.identify_cash_items(cust_trades_df, mapping, remove_cash_items=False, file_type="transactions")
cust_trades_df.tail(4)

,fund_code,fund_name,isin,client_id,security_description,security_group,trd_number,transaction_code,trade_date,settle_date,units,price,principal,trading_currency,country_of_risk,country_of_risk_name,trade_tax,broker_commission,exchange_rate,LUSID.base_currency,LUSID.created,__currency_identifier_for_LUSID
17,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00BH4HKS39,EQ_1242,vodafone,COMMON STOCK,trd_140,cust_buy,27/11/2019,29/11/2019,467990,1.60,750000.77,GBP,GB,United Kingdom,360,30,1,GBP,2018-01-01T00:00:00+00:00,None
18,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,trd_141,cust_buy,27/11/2019,29/11/2019,36452,20.57,749999.90,GBP,GB,United Kingdom,355,20,1,GBP,2018-01-01T00:00:00+00:00,None
19,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,GB00B1XZS820,EQ_1243,Anglo American plc,COMMON STOCK,trd_142,cust_buy,27/11/2019,29/11/2019,36452,20.57,749999.90,GBP,GB,United Kingdom,355,40,1,GBP,2018-01-01T00:00:00+00:00,None
20,LUSID_COM001,LUSID's top 10 FTSE stock portfolio,NaN,NaN,GBP Cash,CASH,trd_143,cust_buy,27/11/2019,29/11/2019,12000000,1.00,12000000.00,GBP,GB,United Kingdom,0,0,1,GBP,2018-01-01T00:00:00+00:00,GBP


In [10]:
result = lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=cust_trades_df,
    mapping_required=mapping["transactions"]["required"],
    mapping_optional=mapping["transactions"]["optional"],
    file_type="transactions",
    identifier_mapping=mapping["transactions"]["identifier_mapping"],
    property_columns=mapping["transactions"]["properties"],
    properties_scope=scope
)

succ, failed = format_transactions_response(result)
print(f"number of successful portfolios requests: {len(succ)}")
print(f"number of failed portfolios requests    : {len(failed)}")

number of successful portfolios requests: 1
number of failed portfolios requests    : 0


## (8) Map the transaction types to movements <a id="trd_movements"></a>  

Next we configure some "movements" which tells LUSID how to build transactions into holdings. First, let us configure a "side" called TradeCommission. In most simple terms, this side is used to capture the broker commission from the "broker_commission" property on a Transaction. Recall also that we created that property from the initial CSV file.

In [11]:
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

side_list = [
        models.SideConfigurationDataRequest(
            side="TradeCommissions",
            security="Txn:LusidInstrumentId",
            currency="Txn:SettlementCurrency",
            rate="Transaction/custodian/exchange_rate",
            units="Transaction/custodian/broker_commission",
            amount="Transaction/custodian/broker_commission"
        )
    ]


for side in side_list:
    
    current_sides = [side.side for side in system_configuration_api.list_configuration_transaction_types().side_definitions]
    
    if side.side in list(current_sides):
        
        print(f"{side.side} already exists in LUSID")
    
    else:
        
        response = system_configuration_api.create_side_definition(side_configuration_data_request = side)
        
        print(f"{side.side} has been created in LUSID")

TradeCommissions has been created in LUSID


Then we create some "transaction type" configurations. These are instructions for LUSID when building holdings from transactions. For example, when LUSID sees that a transaction has a "transaction type" of "cust_buy", it knows it needs to do three things:<br>

(1) Increase the stock in the portfolio.<br> 
(2) Decrease the cash.<br>
(3) Increase the commission paid

In [12]:
response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="lusid_security_services", 
    type="cust_buy",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="cust_buy",
                description="cust_buy",
                transaction_class="cust_buy",
                transaction_roles="None",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types='StockMovement',
                side='Side1',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionTypePropertyMapping(
                        property_key="Transaction/custodian/AssetType",
                        set_to="Stock"
                    )]),
            models.TransactionTypeMovement(
                movement_types='CashCommitment',
                side='Side2',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionTypePropertyMapping(
                        property_key="Transaction/custodian/AssetType",
                        set_to="Cash"
                    )]),
            models.TransactionTypeMovement(
                movement_types='CashCommitment',
                side='TradeCommissions',
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionTypePropertyMapping(
                        property_key="Transaction/custodian/AssetType",
                        set_to="Commission"
                    )])]))

Next, we create a seperate movement for the "custodian_sub" transactions - these are subscriptions which is when a client gives more money to a fund manager for investing. The "custodian_sub" movement is more simple than the buy movements. For a "custodian_sub" movement, we simply increase the cash in a portfolio.

In [13]:
response = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="lusid_security_services", 
    type="custodian_sub",
    transaction_type_request=lusid.models.TransactionTypeRequest(
        aliases=[
            lusid.models.TransactionTypeAlias(
                type="custodian_sub",
                description="custodian_sub",
                transaction_class="custodian_sub",
                transaction_roles="None",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types='CashAccrual',
                side='Side1',
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionTypePropertyMapping(
                    property_key="Transaction/custodian/AssetType",
                        set_to="Cash")
                    ])]))

## (9) Create holdings report <a id="holdings"></a>  

Finally we generate a holdings report in LUSID using the get_holdings endpoint. Based on the configuration above, a number of things happen when we call this endpoint. It's worth going through them one-by-one:
<br>

* First, LUSID gathers the transactions for this portfolio from the events register (these are the transactions which will be used to build holdings).
* Then based on the movements configuration defined above, a transaction will then be used to build a holding as follows.<br>
    * The transactions with a transaction type of "custodian_buy" will increase holdings, decrease cash, and increase the amount of commission paid.
    * The transactions with a transaction type of "custodian_sub" will increase the cash only. This is designed to represent a portfolio subscription.
* In reality there will also be an inverse movement configured for "custodian_buy" and "custodian_sub" (i.e. "custodian_sell" and "custodian_red")

In [14]:
def generate_holdings_df(scope, portfolio):

    holdings_response = api_factory.build(
        lusid.api.transaction_portfolios_api.TransactionPortfoliosApi
    ).get_holdings(scope, portfolio, property_keys=["Instrument/default/Name"])

    holdings_df = lusid_response_to_data_frame(holdings_response)

    holdings_df = holdings_df.rename(
        columns={
            "sub_holding_keys.Transaction/custodian/AssetType.value.label_value": "AssetType",
            "properties.Instrument/default/Name.value.label_value": "name",
            "cost.amount": "Value",
        }
    )

    return holdings_df[["AssetType", "name", "Value"]]

View holdings report:

In [15]:
holdings_df = generate_holdings_df(scope, portfolio)
holdings_df_formatted = holdings_df.copy()
holdings_df_formatted["Value"] = holdings_df_formatted["Value"].map('£{:,.2f}'.format)
holdings_df_formatted

,AssetType,name,Value
0,Stock,Aviva,"£549,997.05"
1,Commission,Aviva,£-50.00
2,Stock,BHP,"£2,000,008.64"
3,Commission,BHP,£-60.00
4,Stock,Barclays,"£500,000.54"
5,Commission,Barclays,£-50.00
6,Stock,BP,"£1,000,002.74"
7,Commission,BP,£-40.00
8,Stock,HSBC,"£249,996.80"
9,Commission,HSBC,£-50.00


## (10) View and track your comissions <a id="com_report"></a>  

We can then subtotal the results.

Here we can see that the portfolio has paid £500 in commission over its lifetime.

In [16]:
holdings_df[["AssetType","Value"]].groupby("AssetType").sum()

,Value
AssetType,
Cash,-21550009.39
Commission,-500.00
Stock,21550009.39
